In [1]:
from pyspark.sql import SparkSession
from pymongo import MongoClient

In [2]:
client = MongoClient('mongo', 27017)

In [3]:
db = client.test
collection = db.dummy_collection

In [4]:
data = [
    {"name": "John", "age": 25},
    {"name": "Jane", "age": 30},
    {"name": "Bob", "age": 35}
]

result = collection.insert_many(data)
print(f"Inserted {len(result.inserted_ids)} documents")

for document in collection.find():
    print(document)

Inserted 3 documents
{'_id': ObjectId('64304fc00281c471e09ef1eb'), 'name': 'John', 'age': 25}
{'_id': ObjectId('64304fc00281c471e09ef1ec'), 'name': 'Jane', 'age': 30}
{'_id': ObjectId('64304fc00281c471e09ef1ed'), 'name': 'Bob', 'age': 35}
{'_id': ObjectId('64305181ec785abdbd310bdc'), 'name': 'John', 'age': 25}
{'_id': ObjectId('64305181ec785abdbd310bdd'), 'name': 'Jane', 'age': 30}
{'_id': ObjectId('64305181ec785abdbd310bde'), 'name': 'Bob', 'age': 35}
{'_id': ObjectId('64305225a414bceed1525031'), 'name': 'John', 'age': 25}
{'_id': ObjectId('64305225a414bceed1525032'), 'name': 'Jane', 'age': 30}
{'_id': ObjectId('64305225a414bceed1525033'), 'name': 'Bob', 'age': 35}
{'_id': ObjectId('64305efda22c97ad59a8b7b5'), 'name': 'John', 'age': 25}
{'_id': ObjectId('64305efda22c97ad59a8b7b6'), 'name': 'Jane', 'age': 30}
{'_id': ObjectId('64305efda22c97ad59a8b7b7'), 'name': 'Bob', 'age': 35}


In [5]:
client.close()

In [6]:
packages = {
   'org.mongodb.spark:mongo-spark-connector_2.12:10.1.1',
}

In [7]:
spark = (
    SparkSession.builder
    .config('spark.jars.packages', ','.join(packages))
    .config("spark.mongodb.input.uri", "mongodb://mongo:27017/test.dummy_collection")
    .config("spark.mongodb.output.uri", "mongodb://mongo:27017/test.dummy_collection")
    .config("spark.mongodb.connection.uri", "mongodb://mongo:27017")
    .config("spark.mongodb.database", "test")
    .config("spark.mongodb.collection", "dummy_collection")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/home/hduser/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hduser/.ivy2/cache
The jars for the packages stored in: /home/hduser/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8eeea6f0-47d4-4de3-bd22-266951ce0c11;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;10.1.1 in central
	found org.mongodb#mongodb-driver-sync;4.8.2 in central
	[4.8.2] org.mongodb#mongodb-driver-sync;[4.8.1,4.8.99)
	found org.mongodb#bson;4.8.2 in central
	found org.mongodb#mongodb-driver-core;4.8.2 in central
	found org.mongodb#bson-record-codec;4.8.2 in central
:: resolution report :: resolve 2772ms :: artifacts dl 36ms
	:: modules in use:
	org.mongodb#bson;4.8.2 from central in [default]
	org.mongodb#bson-record-codec;4.8.2 from central in [default]
	org.mongodb#mongodb-driver-core;4.8.2 from central in [default]
	org.mongodb#mongodb-driver-sync;4.8.2 from central in [default]
	org.mongodb.spark#mongo-spark-connector

23/04/07 18:21:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
people = spark.createDataFrame(
    [
        ("Bilbo Baggins", 50),
        ("Gandalf", 1000),
        ("Thorin", 195),
        ("Balin", 178),
        ("Kili", 77),
        ("Dwalin", 169),
        ("Oin", 167),
        ("Gloin", 158),
        ("Fili", 82),
        ("Bombur", None),
    ],
    ["name", "age"],
)


In [9]:
people.write.format("mongodb").mode("overwrite").save()

In [10]:
df = spark.read.format("mongodb").load()
df.printSchema()
df.show()

root
 |-- _id: string (nullable = true)
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+--------------------+----+-------------+
|                 _id| age|         name|
+--------------------+----+-------------+
|64305f42a842da159...|1000|      Gandalf|
|64305f42a842da159...|null|       Bombur|
|64305f42a842da159...|  77|         Kili|
|64305f42a842da159...| 158|        Gloin|
|64305f42a842da159...| 195|       Thorin|
|64305f42a842da159...| 178|        Balin|
|64305f42a842da159...|  50|Bilbo Baggins|
|64305f42a842da159...|  82|         Fili|
|64305f42a842da159...| 169|       Dwalin|
|64305f42a842da159...| 167|          Oin|
+--------------------+----+-------------+

